In [13]:
from tqdm.notebook import trange, tqdm
from tba_types import Award, Event
from tba_session import session, ROOT_URL, TEAM_PAGES
from collections import defaultdict

In [14]:
ALL_TEAM_KEYS: list[str] = []

for i in trange(TEAM_PAGES, desc="Getting all teams", unit="page"):
    ALL_TEAM_KEYS.extend(
        session.get(f"{ROOT_URL}/teams/{i}/keys").json()
    )

Getting all teams:   0%|          | 0/22 [00:00<?, ?page/s]

In [40]:
team_awards: dict[str, dict[int, list[Award]]] = {}
num_seasons: dict[str, int] = {}

events: set[str] = set()

for year in trange(1993, 2026, desc="Getting events", unit="year"):
    events_this_year: list[Event] = session.get(f"{ROOT_URL}/events/{year}").json()
    events.update(
        event["key"]
        for event in events_this_year
        if 0 <= event["event_type"] <= 7 and "cancelled" not in event["name"].lower()
    )

for team_key in tqdm(ALL_TEAM_KEYS, desc="Getting team awards", unit="team"):
    team_awards[team_key] = defaultdict(list)
    num_seasons[team_key] = len(
        session.get(f"{ROOT_URL}/team/{team_key}/years_participated").json()
    )

    awards: list[Award] = session.get(f"{ROOT_URL}/team/{team_key}/awards").json()
    for award in filter(lambda a: a["event_key"] in events, awards):
        team_awards[team_key][award["award_type"]].append(award)

Getting events:   0%|          | 0/33 [00:00<?, ?year/s]

Getting team awards:   0%|          | 0/8831 [00:00<?, ?team/s]

In [30]:
awards_to_count: dict[str, list[int]] = {
    "Impact": [0, 69],
    "Winner": [1],
    "Finalist": [2],
    "Woodie Flowers": [3],
    "Dean's List": [4],
    "Engineering Inspiration": [9],
    "Rookie All-Star": [10],
    "Gracious Professionalism": [11],
    "Judges'": [13],
    "Industrial Design": [16],
    "Quality": [17],
    "Safety": [18],
    "Creativity": [20],
    "Excellence in Engineering": [21, 26],
    "Entrepreneurship/Sustainability": [22, 82],
    "Imagery": [27],
    "Innovation in Control": [29, 38],
    "Team Spirit": [30],
    "Autonomous": [71],
}


def award_count(team_key: str, to_count: list[int]) -> int:
    return sum(
        len(team_awards[team_key][award_type])
        for award_type in to_count
        if award_type in team_awards[team_key]
    )


def award_count_per_season(team_key: str, to_count: list[int]) -> float:
    if num_seasons[team_key] == 0:
        return 0.0
    return award_count(team_key, to_count) / num_seasons[team_key]


print("| Award | #1 (Ct) | #2 (Ct) | #3 (Ct) |")
print("| --- | --- | --- | --- |")
for award_name, award_types in awards_to_count.items():
    teams = sorted(
        ALL_TEAM_KEYS,
        key=lambda team_key: award_count(team_key, award_types),
        reverse=True,
    )
    print(
        f"| {award_name} | {
            " | ".join(
                f"[{team_key[3:]}](https://thebluealliance.com/team/{team_key[3:]}) ({award_count(team_key, award_types)})"
                for team_key in teams[:3]
            )
        } |"
    )

print()
print("| Award | #1 (Ct/Season) | #2 (Ct/Season) | #3 (Ct/Season) |")
print("| --- | --- | --- | --- |")
for award_name, award_types in awards_to_count.items():
    teams = sorted(
        ALL_TEAM_KEYS,
        key=lambda team_key: (
            award_count_per_season(team_key, award_types),
            award_count(team_key, award_types),
        ) if num_seasons[team_key] > 1 else (0, 0),
        reverse=True,
    )
    print(
        f"| {award_name} | {
            " | ".join(
                f"[{team_key[3:]}](https://thebluealliance.com/team/{team_key[3:]}) ({award_count_per_season(team_key, award_types):.2f})"
                for team_key in teams[:3]
            )
        } |"
    )

| Award | #1 (Ct) | #2 (Ct) | #3 (Ct) |
| --- | --- | --- | --- |
| Impact | [503](https://thebluealliance.com/team/503) (23) | [1086](https://thebluealliance.com/team/1086) (17) | [4125](https://thebluealliance.com/team/4125) (17) |
| Winner | [254](https://thebluealliance.com/team/254) (61) | [2056](https://thebluealliance.com/team/2056) (54) | [118](https://thebluealliance.com/team/118) (51) |
| Finalist | [27](https://thebluealliance.com/team/27) (26) | [67](https://thebluealliance.com/team/67) (21) | [125](https://thebluealliance.com/team/125) (19) |
| Woodie Flowers | [525](https://thebluealliance.com/team/525) (7) | [987](https://thebluealliance.com/team/987) (7) | [604](https://thebluealliance.com/team/604) (6) |
| Dean's List | [2486](https://thebluealliance.com/team/2486) (14) | [1912](https://thebluealliance.com/team/1912) (13) | [118](https://thebluealliance.com/team/118) (11) |
| Engineering Inspiration | [75](https://thebluealliance.com/team/75) (15) | [141](https://thebl

In [ ]:
if "get_ipython" not in globals():
    import json
    import pathlib

    output_dir = pathlib.Path(__file__).parent / "build"

    def config(count_type: str, title: str) -> dict:
        return {
            "columns": [
                {
                    "name": "team",
                    "type": "team",
                    "displayName": "Team",
                },
                *(
                    {
                        "name": award_name.lower().replace(" ", "_"),
                        "type": count_type,
                        "displayName": award_name,
                    }
                    for award_name in awards_to_count
                ),
            ],
            "primaryKey": "team",
            "defaultSort": [{"column": "team", "direction": 1}],
            "title": title,
        }

    with (output_dir / "award-count.config.json").open("w") as f:
        json.dump(
            config("integer", "Award Counts"),
            f,
            indent=None,
        )
    with (output_dir / "award-count.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    **{
                        award_name.lower().replace(" ", "_"): award_count(
                            team_key, award_types
                        )
                        for award_name, award_types in awards_to_count.items()
                    },
                }
                for team_key in ALL_TEAM_KEYS
            ],
            f,
            indent=None,
        )
    with (output_dir / "award-count-per-season.config.json").open("w") as f:
        json.dump(
            config("float", "Award Counts per Season"),
            f,
            indent=None,
        )
    with (output_dir / "award-count-per-season.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    **{
                        award_name.lower().replace(" ", "_"): award_count_per_season(
                            team_key, award_types
                        )
                        for award_name, award_types in awards_to_count.items()
                    },
                }
                for team_key in ALL_TEAM_KEYS
            ],
            f,
            indent=None,
        )